# A05_LLaMA-Factory-Cmd-lora_single_gpu

## 初始環境設定
- 重新reload kernel, 並切換到 kernel Image_S-work-LlamaFactory_c00cjz00_cuda11.8_pytorch_2.1.2-cuda11.8-cudnn8-devel

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
# GPU確認
!nvidia-smi
import torch
torch.cuda.is_available()

In [ ]:
# 套件安裝 (此映像檔已經安裝以下套件, 若使用其他映像檔, 請移除 ## 安裝以下套件)
##!pip install llmtuner==0.5.3 deepspeed==0.13.1  bitsandbytes==0.42.0 -q 
##!pip install deepspeed==0.13.1 -q
##!pip install bitsandbytes==0.42.0 -q

## SFT (lora in single GPU)

In [ ]:
%%bash
## qlora in single GPU (bitsandbytes) 
## 啟動 Llama-Factory Cmd
## 請記得修改最後一行, 給予huggingface token,  HF_TOKEN='hf_' ?
## 記錄檔案為 llamafactory.log 
## 量化請加上     --quantization_bit 4 \
## --model_name_or_path meta-llama/Llama-2-7b-hf \
## --model_name_or_path /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf \
## --model_name_or_path ../saves/LLaMA2-7B/lora/pretrain_model

## 清空之前計算
ps -ef |grep 'train_bash.py' | awk '{print $2}' | xargs kill -9
sleep 1
rm -rf saves/LLaMA2-7B/lora/sft
sleep 1

cd LLaMA-Factory

CUDA_VISIBLE_DEVICES=0 HF_TOKEN='hf_' python src/train_bash.py \
    --stage sft \
    --do_train \
    --model_name_or_path /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf \
    --dataset alpaca_gpt4_en,glaive_toolcall \
    --dataset_dir data \
    --template default \
    --finetuning_type lora \
    --lora_target q_proj,v_proj \
    --output_dir ../saves/LLaMA2-7B/lora/sft \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --val_size 0.1 \
    --plot_loss \
    --fp16